# SYS 611: Dice Fighters Example (w/ Binomial Process Gen.)

Paul T. Grogan <pgrogan@stevens.edu>

This example shows how to model the dice fighters example in Python using a binomial process generator.

## Dependencies

This example is compatible with Python 2 environments through use of the `__future__` library function. Additionally, this example uses the `numpy` and `scipy.stats` libraries.

In [21]:
# import the python3 behavior for importing, division, and printing in python2
from __future__ import absolute_import, division, print_function

# import the numpy library and refer to it as `np`
import numpy as np

# import the scipy.stats library and refer to it as `stats`
import scipy.stats as stats

## Elementary State Variables

There are five elementary state variables defined below:
 * `round_number`: Current round number
 * `red_size`: Red force size 
 * `blue_size`: Blue force size
 * `red_chance_hit`: Red team probability of landing a 'hit' on a blue team
 * `blue_chance_hit`: Blue team probability of landing a 'hit' on a red team

All variables are defined with global scope and initialized to an initial value.

A helper function `print_state` formats the display of key state variables.

In [22]:
round_number = 0
red_size = 20
blue_size = 10
red_chance_hit = 1/6
blue_chance_hit = 3/6

def print_state():
    print("Round: {:d} | Red: {:d}, Blue: {:d}".format(round_number, red_size, blue_size))

## Derived State Variables

There is one derived state variable defined below:
 * `is_complete`: Determines if a game is complete.

In [23]:
def is_complete():
    """
    Check if the game is complete, meaning at least one team has no forces remaining.
    Return True if the game is complete, False otherwise.
    """
    return (red_size <= 0 or blue_size <= 0)

## Process Generators

There are two process generator functions defined below:
 * `generate_red_hits`: a process generator to determine how many hits the red team scores
 * `generate_blue_hits`: a process generator to determine how many hits the blue team scores

These functions use the binomial inverse CDF function (called a PPF function in `scipy.stats`) following the inverse transform method (IVT) to generate the number of hits based on the number of forces remaining.

In [24]:
# define the generate_red_hits function
def generate_red_hits():
    """
    Randomly generate the number of red hits on the blue team.
    """
    # use the binomial PPF (inverse CDF) with a random sample and cast to an integer
    return int(stats.binom.ppf(np.random.rand(), red_size, red_chance_hit))
    # note: the code above could be replaced by a built-in numpy process generator:
    # return np.random.binomial(red_size, red_chance_hit)

# define the generate_blue_hits function
def generate_blue_hits():
    """
    Randomly generate the number of blue hits on the red team.
    """
    # use the binomial PPF (inverse CDF) with a random sample and cast to an integer
    return int(stats.binom.ppf(np.random.rand(), blue_size, blue_chance_hit))
    # note: the code above could be replaced by a built-in numpy process generator:
    # return np.random.binomial(blue_size, blue_chance_hit)

## State Transition Functions

There are three state transition functions defined below:
 * `red_suffer_losses`: decreases the red force size by the number of blue hits
 * `generate_red_hits`: decreases the blue force size by the number of red hits
 * `next_round`: advances to the next round

In [25]:
def red_suffer_losses(opponent_hits):
    """
    Decrease the red team size by the number of blue hits.
    """
    # (note: red_size must be declared as a global variable to update in this function!)
    global red_size
    # update the red_size based on the number of opponent hits
    red_size -= opponent_hits

def blue_suffer_losses(opponent_hits):
    """
    Decrease the blue team size by the number of red hits.
    """
    # (note: blue_size must be declared as a global variable to update in this function!)
    global blue_size
    # update the blue_size based on number of opponent hits
    blue_size -= opponent_hits

def next_round():
    """
    Advance to the next round.
    """
    # (note: round_number must be declared as a global variable to update in this function!)
    global round_number
    # advance the round_number
    round_number += 1

## Simulation Execution

The following script runs a complete dice fighters match.

In [26]:
round_number = 0
red_size = 20
blue_size = 10
red_chance_hit = 1/6
blue_chance_hit = 3/6

# main execution loop: continue while the game is not complete
while not is_complete():
    # generate the number of red hits
    red_hits = generate_red_hits()
    # generate the number of blue hits
    blue_hits = generate_blue_hits()
    # red team suffers losses of blue hits
    red_suffer_losses(blue_hits)
    # blue team suffers losses of red hits
    blue_suffer_losses(red_hits)
    # advance to the next round
    next_round()
    # print out the current state for debugging
    print_state()
    
# after main loop exists, check who won (whichever team still has fighters!)
if red_size > 0:
    print("Red Wins")
elif blue_size > 0:
    print("Blue Wins")
else:
    print("Tie - Mutual Destruction!")

Round: 1 | Red: 15, Blue: 6
Round: 2 | Red: 14, Blue: 5
Round: 3 | Red: 11, Blue: 3
Round: 4 | Red: 10, Blue: 1
Round: 5 | Red: 10, Blue: -1
Red Wins
